## Run with pub2tools output

1. Define run settings:

    * _include_preprints_ (bool): set to False if preprints should not be included in the tools to add to the database.
    * _to_curate_ (int || 'all'): number of published tools to be added to the database.

In [3]:
include_preprints = False
to_curate = 100

2. Define file paths:

    * _json_file_: path for json file with output from Pub2Tools.
    * _pub2tools_log_:

In [2]:
json_file = "C:/Users/aname/Downloads/to_biotools.json"
pub2tools_log = "pub2tools.log"

3. Define username and password.

In [3]:
username = 'curation'
password = 'D9v7#5KV'

4. Authentication.

In [4]:
from biotools_dev import login_prod

token = login_prod(username, password)

5. Read pub2tools output and get tools with high confidence score from json file.

In [5]:
import json
from tool_processing import process_tools

with open(json_file) as jf:
    data = json.load(jf)
    tools = data['list']

processed_tools = process_tools(tools)

**Tool validation**

6. Validate tools and separate them into valid and problem tools.

In [6]:
from tool_validation import validate_tools

valid_tools, problem_tools = validate_tools(processed_tools, token)

Tool with name:HISTA has the errors: {"biotoolsID":["A resource with this ID already exists. bio.tools IDs need to be unique"]}
Tool with name:Trackplot has the errors: {"biotoolsID":["A resource with this ID already exists. bio.tools IDs need to be unique"]}
Tool with name:SpatialDDLS has the errors: {"biotoolsID":["A resource with this ID already exists. bio.tools IDs need to be unique"]}
Tool with name:aenmd has the errors: {"biotoolsID":["A resource with this ID already exists. bio.tools IDs need to be unique"]}
Tool with name:VoDEx has the errors: {"biotoolsID":["A resource with this ID already exists. bio.tools IDs need to be unique"]}
Tool with name:OBIA has the errors: {"biotoolsID":["A resource with this ID already exists. bio.tools IDs need to be unique"]}
Tool with name:TreeProfiler has the errors: {"biotoolsID":["A resource with this ID already exists. bio.tools IDs need to be unique"]}
Tool with name:KidneyGPS has the errors: {"biotoolsID":["A resource with this ID already

In [1]:
from utils.json_utils import load_tools_from_json

valid_tools = load_tools_from_json('process_scrap_2023_10/low_tools_prp_2023_10.json')

In [9]:
valid_tools

[{'name': 'BayesKAT',
  'description': 'Bayesian Optimal Kernel-based Test for genetic association studies reveals joint genetic effects in complex diseases.\n\nABSTRACT GWAS methods have identified individual SNPs significantly associated with specific phenotypes. Nonetheless, many complex diseases are polygenic and are controlled by multiple genetic variants that are usually non-linearly dependent. These genetic variants are marginally less effective and remain undetected in GWAS analysis. Kernel-based tests (KBT), which evaluate the joint effect of a group of genetic variants, are therefore critical for complex disease analysis. However, choosing different kernel functions in KBT can significantly influence the type I error control and power, and selecting the optimal kernel remains a statistically challenging task. A few existing methods suffer from inflated type 1 errors, limited scalability, inferior power, or issues of ambiguous conclusions',
  'homepage': 'https://github.com/wa

**Identify preprints**

7. Identify preprints and get back the same list of tools including _is_preprint_ flag and _publication_link_ for each tool.

In [2]:
from preprints import identify_preprints

updated_tools = identify_preprints(rerun = False, tools = valid_tools)

There are 8 newly published tools. 112 preprints remaining.


**Create .csv file**

8. Generate csv file from _to_curate_ first publications and all preprints if _include_preprints_ = True.

    Returns:
    
    * _tools_to_add_: tools to add to database (including preprints if _include_preprints_ = True).
    * _tools_left_: tools not in _tools_to_add.

In [4]:
from utils.utils import check_date
from utils.csv_utils import generate_csv

file_date = check_date(pub2tools_log)
tools_to_add, tools_left = generate_csv(updated_tools, to_curate, file_date, include_preprints)


NameError: name 'pub2tools_log' is not defined

**Create json files**

9. Generate json files with tools that will not be curated:

    If _separate_preprints_ = True, 2 json files will be generated with:
    * Preprints (can be used as input for identify_preprints later as rerun = True)
    * Publications

    If _separate_preprints_ = False, only 1 file will be generate with the publications and preprints in the same file.

In [ ]:
from utils.json_utils import generate_json

generate_json(tools_left, file_date, separate_preprints=True)

10. **Add tools to curate to dev**

In [ ]:
from biotools_dev import add_tools

add_tools(tools_to_add, token, WRITE_TO_DB=True)


## Rerun

Workflow to rerun preprints to check for new publications.

Newly published tools in _json_preprints_ will be moved from this file to _json_publications_.

1. Define file paths for preprints and publications files:

In [ ]:
json_preprints = ""
json_publications = ""

2. Run identify_preprints

In [ ]:
from preprints import identify_preprints

updated_tools = identify_preprints(rerun = True, json_prp = json_preprints, json_pub = json_publications)

## Run Pub2Tools output and rerun existing preprints